In [ ]:
import numpy as np
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

from utils.base_directory import BASE_DIRECTORY
from utils.datasets import get_datasets

In [ ]:
DATASET_SAMPLE_SIZE = 1000


def systematic_sample(group, n):
    indexes = list(range(0, len(group), max(1, len(group) // n)))[:n]
    return group.iloc[indexes]


datasets = []
dataset_names = []
for dataset in get_datasets():
    dataframe = dataset.dataframe
    dataset_names.append(dataset.name)

    n_samples = int(np.ceil(DATASET_SAMPLE_SIZE / dataframe["leaning"].nunique()))
    dataframe = (dataframe.groupby("leaning", group_keys=False, observed=True)[["body", "leaning"]]
                 .apply(lambda group: systematic_sample(group, n_samples))
                 .head(DATASET_SAMPLE_SIZE))

    dataframe = dataframe.rename(columns={"leaning": "label"})
    dataframe["label"] = dataframe["label"].cat.rename_categories({"left": 0, "center": 1, "right": 2})

    datasets.append(Dataset.from_pandas(dataframe))

In [ ]:
MODELS_MAX_LENGTH = 512

model_names = ["google-bert/bert-base-cased", "FacebookAI/roberta-base"]

for model_name in model_names:
    print(f"fine-tuning {model_name} on:")

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
    for dataset, dataset_name in zip(datasets, dataset_names):
        print(f"  {dataset_name}")
        output_directory = (BASE_DIRECTORY
                            / "models_custom"
                            / "dataset_benchmark"
                            / model_name.split("/")[-1]
                            / dataset_name)

        tokenized_dataset = dataset.map(
            lambda batch: tokenizer(batch["body"], max_length=MODELS_MAX_LENGTH, truncation=True, padding="max_length"),
            batched=True
        )
        training_arguments = TrainingArguments(
            auto_find_batch_size=True,
            save_strategy="no",
            output_dir=output_directory
        )
        trainer = Trainer(
            model=model,
            args=training_arguments,
            train_dataset=tokenized_dataset
        )
        trainer.train()
        trainer.save_model(output_directory)